In [1]:
!nvidia-smi

Wed Oct 30 02:00:21 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   75C    P0             45W /   80W |    1968MiB /   8192MiB |     17%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
%pip install cucim-cu12 cupy-cuda12x

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 22.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.1/515.1 MB 34.0 MB/s eta 0:00:0000:0100:01


In [ ]:
%load_ext cudf.pandas

from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
import cupy as cp
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob

print("Is torch using cuda? ",torch.cuda.is_available())
print("Is cupy using cuda? ",cp.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


2024-10-29 11:55:07.796061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 11:55:07.814395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 11:55:07.819805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 11:55:07.832856: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Is torch using cuda?  True
Is cupy using cuda?  True
Is tensorflow using cuda?  True
Is pandas using cuda?  <module 'pandas' (ModuleAccelerator(fast=cudf, slow=pandas))>


In [ ]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [ ]:
def sort_by_number(texts:list[str]):
    def key(text:str):
        return int(re.search(r'\d+', text).group())
    return sorted(texts, key=key)

In [ ]:
base_path = '/home/workspace/geometry-classifier/data/'

csv_files = sort_by_number(glob.glob(base_path+'photos_v2/*.csv'))

chunck_size = 4
predictions = []
# Batch processing and prediction
batch_size = 32
true_labels = []
ids = []

model = tf.keras.applications.ResNet50(weights='imagenet', input_shape=(224, 224, 3), include_top=True)

def current_chunk(df):
    labels = df['name'].values
    images = df.drop(columns=['name', 'id'])

    # Preprocess function (as before)
    def preprocess_cudf_df(images_df, labels, batch_size):
        # Convert images to numpy and reshape for model input
        df_np = images_df.values.reshape(-1, 224, 224, 1)
        df_rgb = np.repeat(df_np, 3, axis=-1)
        
        # Create batches
        num_samples = df_rgb.shape[0]
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            yield df_rgb[start:end], labels[start:end]

    for batch_images, batch_labels in preprocess_cudf_df(images, labels, batch_size):
        print("Processing...")
        preds = model.predict(batch_images)
        predicted_classes = np.argmax(preds, axis=1)
        predictions.extend(predicted_classes)
        true_labels.extend(batch_labels)
    
    ids.extend(df['id'].values)


I0000 00:00:1730202911.541184    2032 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730202911.549798    2032 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730202911.550107    2032 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730202911.551473    2032 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [ ]:
save_to_csv = base_path + 'predictions.csv'
def save_prediction():
    with open(save_to_csv, 'w+') as f:
        f.write('id,prediction,true\n')
        for (id, pred, true) in zip(ids, predictions, true_labels):
            f.write(f"{id},{pred},{true}\n")
        

In [ ]:
import gc
# read chunk of 4 csv files at a time
for csv_file in csv_files:
    print(f"Processing file: {csv_file}")
    df = pd.read_csv(csv_file)
    print("generated dataframe")
    current_chunk(df)
    save_prediction()
    del df
    gc.collect()

Processing file: /home/workspace/geometry-classifier/data/photos_v2/photos_118.csv
generated dataframe
Processing...


I0000 00:00:1730203021.071424    2189 service.cc:146] XLA service 0x780f6c14a3f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730203021.071459    2189 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2024-10-29 11:57:01.154873: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-29 11:57:01.718552: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


I0000 00:00:1730203025.329294    2189 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processing..

KeyboardInterrupt: 

In [ ]:
stringfy_results = [name_mapping[p] + str(t) for p, t in zip(predictions, true_labels)]

plt.figure(figsize=(20, 40))
plt.hist(stringfy_results)
plt.show()

In [ ]:
# cofusion matrix
confusion_matrix = tf.math.confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.show()